In [74]:
from web_scraper.web_scraper import CompendiumScraper
import pandas as pd
from selenium.webdriver.common.by import By
import psycopg2
import subprocess
import yaml
import sqlalchemy as sa
import time
import re
import requests
import json

from selenium.webdriver.common.action_chains import ActionChains

In [59]:
with open(r'\Users\jasre\Code\ws_config.yml', 'r') as yml:
    config = yaml.safe_load(yml)

# Running To Do
- Parse follow ups for JP versions of characters (manual...)
- Add notes for characters:
  - Deuce -- Her Sforzando buff isn't included in the BT aura and will have do be added in the visual
- Set up marts
- Set up Streamlit

In [3]:
cs = CompendiumScraper('\\Users\\jasre\\Code\\ws_config.yml')

2023-09-09 07:54:15,187 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:15,187 - web_scraper.web_scraper - Retrieved all main character links. Generating remaining links for each character.
2023-09-09 07:54:15,187 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:15,248 - web_scraper.web_scraper - self.character_dict_omnibus entry for KELGER was successful
2023-09-09 07:54:15,276 - web_scraper.web_scraper - self.character_dict_omnibus entry for JIHLNABAAT was successful
2023-09-09 07:54:15,291 - web_scraper.web_scraper - self.character_dict_omnibus entry for WEISS was successful
2023-09-09 07:54:15,366 - web_scraper.web_scraper - self.character_dict_omnibus entry for ASTOS was successful
2023-09-09 07:54:15,396 - web_scraper.web_scraper - self.character_dict_omnibus entry for IRIS was successful
2023-09-09 07:54:15,433 - web_scraper.web_scraper - self.character_dict_omnibus entry for REKS 

In [4]:
char_name = 'cecilpaladin'

bt_df = cs.retrieve_hp_caps_from_bt(char_name=char_name, JP=True, return_output=True)

bt_df

2023-09-09 07:54:18,627 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:18,627 - web_scraper.web_scraper - Retrieving BT info for CECILPALADIN.
2023-09-09 07:54:18,627 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:31,583 - web_scraper.web_scraper - Standard BT stack slider div-class not found. Trying second possibility.
2023-09-09 07:54:31,599 - web_scraper.web_scraper - No stack slider found. Assuming CECILPALADIN has a BT without stacks.
2023-09-09 07:54:31,672 - web_scraper.web_scraper - Retrieved BT info for CECILPALADIN.


,char_name,bt_personal_hp_dmg_cap_up,bt_party_hp_dmg_cap_up,enemy_count_apply_list,game_version
0,cecilpaladin,0,50,"[1, 2, 3]",JP


In [5]:
ability_df = cs.generate_ability_df(char_name=char_name, JP=True)

2023-09-09 07:54:31,715 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:31,716 - web_scraper.web_scraper - Parsing ability dict to df for CECILPALADIN.
2023-09-09 07:54:31,717 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:31,718 - web_scraper.web_scraper - Couldn't find ability_dict for CECILPALADIN. Creating it now.
2023-09-09 07:54:31,718 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:31,720 - web_scraper.web_scraper - Generating ability dictionary for CECILPALADIN
2023-09-09 07:54:31,721 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:38,065 - web_scraper.web_scraper - This iteration caught 4 abilities.
2023-09-09 07:54:38,065 - web_scraper.web_scraper - -----------
2023-09-09 07:54:39,096 - web_scraper.web_scraper - This iteration caught 4 abilities.
2023-09-09 07:54:39,096 - web_sc

In [6]:
ability_df

,char_name,ability_name,ability_id,main_target_hp_attacks,non_target_hp_attacks,hp_dmg_cap_up_perc,attribute_list,game_version
0,cecilpaladin,Light Slash,17119,1,0,20,"[Melee, Holy]",JP
1,cecilpaladin,Luminous Blast,17121,2,1,20,"[Ranged, Holy]",JP
2,cecilpaladin,Saint's Fall,21415,10,1,20,"[Melee, Holy]",JP
3,cecilpaladin,Holy Flame,21417,6,6,20,"[Group, Ranged, Holy, Heals]",JP
4,cecilpaladin,Light Enchant All,17071,0,0,0,"[Instant, NoSummon, NoEX]",JP
5,cecilpaladin,Paladin Force,21418,8,8,10,"[Melee, Holy]",JP
6,cecilpaladin,Luminous Shard,21419,8,8,15,"[Group, Ranged, Holy, Heals, NoFree]",JP
7,cecilpaladin,Soul Shift,21421,2,2,350,"[Group, Ranged, Holy]",JP
8,cecilpaladin,Spark of Saint,21426,4,4,250,"[Group, Ranged, Heals, NoFree]",JP
9,cecilpaladin,Enhanced Holy Flame (C),11254,1,1,0,"[Group, Ranged, Holy, Heals, Instant, NoEX, No...",JP


In [7]:
ha_df = cs.retrieve_ha_hp_dmg_cap_up(char_name=char_name, JP=True, return_output=True)

ha_df

2023-09-09 07:54:43,846 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:43,846 - web_scraper.web_scraper - Retrieving High Armor info for CECILPALADIN.
2023-09-09 07:54:43,846 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:54,733 - web_scraper.web_scraper - High armor info parsed for CECILPALADIN.


,char_name,personal_hp_dmg_cap_up,party_ha_hp_dmg_cap_up,game_version
0,cecilpaladin,10,10,JP


# Following can be used to manually upload data to Postgres if the web scraper fails to do so. 
## DO NOT DELETE!

In [ ]:
engine_url = sa.URL.create(
        "postgresql",
        username=config['pg_user'],
        password=config['pg_pass'],
        host=config['pg_host'],
        database=config['pg_db']
    )
    
engine = sa.create_engine(engine_url)

bt_effects = pd.read_csv(config['datasets_dir'] + 'raw_bt_effects.csv')
abilities = pd.read_csv(config['datasets_dir'] + 'raw_abilities.csv')
ha_caps = pd.read_csv(config['datasets_dir'] + 'raw_high_armor_caps.csv')

with engine.begin() as conn:
    abilities.to_sql('raw_abilities', con=conn, index=False, if_exists='append')
    bt_effects.to_sql('raw_bt_effects', con=conn, index=False, if_exists='append')
    ha_caps.to_sql('raw_high_armor_caps', con=conn, index=False, if_exists='append')

# Set up followup spreadsheet

In [65]:
followup_df = pd.read_csv(r'\Users\jasre\Code\dffoo-data-pipeline\datasets\followup_ability_df.csv', encoding='WINDOWS-1252')

In [66]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

followup_df

,char_name,ability,timing,triggers,default,during_bt,double_during_bt
0,reks,Remove Obstacle,before,all,F,T,T
1,reks,Remove Obstacle+,before,all,T,T,T
2,jackgarland,Soul Burst,during,damaging,F,T,F
3,jackgarland,Soul Burst+,during,damaging,T,T,F
4,dorgann,Image Slash,after,"damaging, ally_attack",F,T,F
5,dorgann,Image Slash+,after,"damaging, ally_attack",T,T,F
6,minwu,Additional attack from Sublime Soul,after,effect,T,T,F
7,lunafreya,Awakening Follow Up,during,Awakening,T,T,F
8,braska,Jecht Beam,after,all,F,T,F
9,cissnei,Besiege,after,"damaging, ally_attack",F,T,F


In [11]:
raw_abilities_df = pd.read_csv(r'\Users\jasre\Code\dffoo-data-pipeline\datasets\raw_abilities.csv')

In [43]:
followup_df['ability'] = followup_df['ability'].apply(lambda x: x.replace('Ã±', 'ñ').replace('â', '–'))

In [46]:
followup_df['triggers'] = followup_df['triggers'].apply(lambda x: x.replace('', '–'))

In [49]:
followup_df

,char_name,ability,timing,triggers,default,during_bt,double_during_bt
0,reks,Remove Obstacle,before,all,F,T,T
1,reks,Remove Obstacle+,before,all,T,T,T
2,jackgarland,Soul Burst,during,damaging,F,T,F
3,jackgarland,Soul Burst+,during,damaging,T,T,F
4,dorgann,Image Slash,after,"damaging, ally_attack",F,T,F
5,dorgann,Image Slash+,after,"damaging, ally_attack",T,T,F
6,minwu,Additional attack from Sublime Soul,after,effect,T,T,F
7,lunafreya,Awakening Follow Up,during,Awakening,T,T,F
8,braska,Jecht Beam,after,all,F,T,F
9,cissnei,Besiege,after,"damaging, ally_attack",F,T,F


In [32]:
new_followup_df = raw_abilities_df[
    raw_abilities_df['attribute_list'].str.contains('FollowUp')
    | raw_abilities_df['attribute_list'].str.contains('Traps')
]

In [68]:
new_followup_df[new_followup_df['char_name'] == 'yuri']

,char_name,ability_name,ability_id,main_target_hp_attacks,non_target_hp_attacks,hp_dmg_cap_up_perc,attribute_list,game_version,scrape_started_at_utc,scrape_ended_at_utc
1508,yuri,Magic Blast – Holyga (Triggered),15733.0,3,3,10,"['Group', 'Magic', 'Holy', 'Instant', 'Traps',...",GL,2023-09-09 11:56:19,2023-09-09 14:05:34
1509,yuri,Magic Blast – Holyga+ (Triggered),17863.0,8,8,10,"['Group', 'Magic', 'Holy', 'Instant', 'Traps',...",GL,2023-09-09 11:56:19,2023-09-09 14:05:34


In [54]:
joined_df = new_followup_df.merge(
    how='outer',
    right=followup_df,
    left_on=['char_name', 'ability_name'],
    right_on=['char_name', 'ability']
)

In [61]:
joined_df.to_csv(r'\Users\jasre\Code\dffoo-data-pipeline\datasets\followups_updated.csv', encoding='utf-8', index=False)

# The following cell will upload your manually-entered followups df into Postgres, including with the correct encodings. DO NOT DELETE.

In [64]:
engine_url = sa.URL.create(
        "postgresql",
        username=config['pg_user'],
        password=config['pg_pass'],
        host=config['pg_host'],
        database=config['pg_db']
    )
    
engine = sa.create_engine(engine_url)

followups_manual_entry = pd.read_csv(r'\Users\jasre\Code\dffoo-data-pipeline\datasets\followups_manual_entry.csv', encoding='WINDOWS-1252')

with engine.begin() as conn:
    followups_manual_entry.to_sql('raw_followups_manual_entry', con=conn, index=False, if_exists='append')

# This cell contains code to load the command abilities JSON blobs into Postgres. DO NOT DELETE.

In [142]:
current_time = str(time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime()))

link_dict = {
    'GL': 'https://dissidiacompendium.com/data/_dir/command_ability/GLNew.json',
    'JP': 'https://dissidiacompendium.com/data/_dir/command_ability/JPNew.json'
}

command_abilities_link_gl = 'https://dissidiacompendium.com/data/_dir/command_ability/GLNew.json'
command_abilities_link_jp = 'https://dissidiacompendium.com/data/_dir/command_ability/JPNew.json'

df_list = []

for game_version, link in link_dict.items():

    response = requests.get(link)

    if response.status_code == 200:

        json_content = response.json()
        
        for ability_id in json_content:
            new_dict = {}
        
            new_dict['ability_id'] = int(ability_id)
            new_dict['rank'] = str(json_content[ability_id].get('rank', None))
            new_dict['name'] = str(json_content[ability_id].get('name', None))
            new_dict['game_version'] = game_version
        
            new_df = pd.DataFrame([new_dict])
        
            df_list.append(new_df)

ability_rank_df = pd.concat(df_list)

ability_rank_df['extracted_at_utc'] = current_time

engine_url = sa.URL.create(
        "postgresql",
        username=config['pg_user'],
        password=config['pg_pass'],
        host=config['pg_host'],
        database=config['pg_db']
    )
    
engine = sa.create_engine(engine_url)

with engine.begin() as conn:
    ability_rank_df.to_sql('raw_ability_ranks', con=conn, index=False, if_exists='replace')